In [26]:
import os

In [39]:
for m in get_m_files():
    ipynb = os.path.splitext(m)[0]+"_07.ipynb"
    nb = convert_m_script_to_ipython_notebook(m)
    with open(ipynb, 'w') as f:
        if False:
            nb_executed, resources = pp.preprocess(nb, resources={})
            print(nbf.writes(nb_executed), file=f)
        else:
            print(nbf.writes(nb), file=f)

A = numpy.array([[-0.313,56.7,0],[-0.0139,-0.426,0],[0,56.7,0]])
B = numpy.array([[0.232],[0.0203],[0]])
C = numpy.array([[0,0,1]])
D = numpy.array([[0]])
A = [-0.313 	56.7 	0;
     -0.0139   -0.426 	0;
      0 	56.7 	0];	
B = [0.232;
     0.0203;
     0];   
C = [0 0 1];
D = [0];
sys_ss = ss(A,B,C,D);
Ts = 1/100;
sys_d = c2d(sys_ss,Ts,'zoh')

co = ctrb(sys_d);
Controllability = rank(co)

A = sys_d.a;
B = sys_d.b;
C = sys_d.c;
D = sys_d.d;
p = 50;
Q = p*C'*C
R = 1;
[K] = dlqr(A,B,Q,R)

time = 0:0.01:10;
theta_des = 0.2*ones(size(time));
sys_cl = ss(A-B*K,B,C,D,Ts);
[y,t] = lsim(sys_cl,theta_des);
stairs(t,y)
xlabel('time (sec)');
ylabel('pitch angle (rad)');
title('Closed-Loop Step Response: DLQR');

Nbar = 6.95;
sys_cl = ss(A-B*K,B*Nbar,C,D,Ts);
[y,t] = lsim(sys_cl,theta_des);
stairs(t,y)
xlabel('time (sec)');
ylabel('pitch angle (rad)');
title('Closed-Loop Step Response: DLQR with Precompensation');
t = numpy.arange(0,10,0.01)
t = [0:0.01:10];
s = tf('s');
P_pitch = (1.151*s + 0.1774

In [36]:
import numpy
import re
def cell_cleaner_upperer(notebook_cell):
    source = notebook_cell["source"]
    if notebook_cell["cell_type"] is "markdown":
        source=markdown_cleaner_upperer(source)
    if notebook_cell["cell_type"] is "code":
        source=matlab_to_python_code_converter(source)
        print(source)
    #print(source, end="")
    notebook_cell["source"] = source
    return notebook_cell
def markdown_cleaner_upperer(markdown_block):
     # Non inline latex
    markdown_block=re.sub(r"\$\$([^$]+)\$\$",r"\n\n$$\n\1\n$$\n\n",markdown_block)
    # Embedded Functions
    markdown_block=re.sub(r"\|([\w_]+)\|",r"```\1```",markdown_block)
    # Links to other
    markdown_block=re.sub(r"< \?example=([\w]+)&section=([\w]+) ([^>]+)>",
                  r"[\3](/examples/\1/\1-\2.ipynb)",markdown_block)
    return markdown_block

In [38]:
def matlab_to_python_code_converter(code_block):
    assignments = re.compile("([\w_]+) ?= ? \[([^]]+)\]").findall(code_block)
    assignment_blocks=list()
    for assignment in assignments:
        variable=assignment[0]
        sin=Homunculus(assignment[1],assignment[0])
        assignment_blocks.append(str(sin))
    return "\n".join(assignment_blocks)+"\n"+code_block

In [23]:
# http://fma.wikia.com/wiki/Homunculus
class Homunculus(object):
    def __init__(self, matlab_line, variable):
        self.matlab_line=matlab_line.strip()
        self.python_line=matlab_line
        self.variable = variable
        self.alchemy()
        
    def alchemy(self):
        value = self.matlab_line
        value=value.replace("\t","")
        value=value.replace("\n","")
        for q in range(10): 
            value=value.replace("  "," ")
        values = [V.replace(" ",",") for V in value.split("; ")]
        if len(values)==1:
            values = values[0].split(";")
        
        pythonic = "numpy.array([["+("],[".join(values)+"]])")
        
        regex=r"""numpy.array\(\[\[([\d.]+):([\d.]+):([\d.]+)\]\]\)"""
        match = re.compile(regex).findall(pythonic)
        if len(match) == 1:
            start, step, end = match[0]
            pythonic = "numpy.arange({start},{end},{step})".format(start=start,end=end,step=step)
        regex=r"""numpy.array\(\[\[([\d.]+):([\d.]+)\]\]\)"""
        match = re.compile(regex).findall(pythonic)
        if len(match) == 1:
            start, end = match[0]
            pythonic = "numpy.arange({start},{end})".format(start=start,end=end,step=step)
        try:
            eval(pythonic)
        except NameError:
            pass
        except:
            if False:
                raise
            else:
                pythonic = "# "+pythonic
        self.python_line = pythonic
        
    def __repr__(self):
        return self.variable+" = "+self.python_line

In [29]:
from nbconvert.preprocessors.execute import ExecutePreprocessor
import os 
try:
    from os import scandir, walk
except ImportError:
    from scandir import scandir, walk
from nbformat import v4 as nbf
import re
def get_m_files(d="."):
    exts=["m"]
    for root, dirs, files in walk(d, topdown=False):
        for name in files:
            if "code_to_convert" in name:
                continue
            if os.path.splitext(name)[1].strip('.') in exts:
                yield os.path.abspath(os.path.join(root, name))
class matlab_to_python_line_transmogrification(object):
    HEADER=0
    COMMENT=1
    CODE=2
    MARKDOWN=3
    """ trans·mog·ri·fied
        trans·mog·ri·fy·ing
        transitive verb
        to change or alter greatly and often with grotesque or humorous effect
    """
    def __init__(self, line):
        self.line = line
        # print("B. "+line)
        if line.startswith("%%"):
            self.type=matlab_to_python_line_transmogrification.HEADER
            self.cell_type = matlab_to_python_line_transmogrification.MARKDOWN
        elif line.startswith("%"):
            self.type=matlab_to_python_line_transmogrification.COMMENT
            self.cell_type = matlab_to_python_line_transmogrification.MARKDOWN
        else:
            self.type=matlab_to_python_line_transmogrification.CODE
            self.cell_type = matlab_to_python_line_transmogrification.CODE
            
    def __repr__(self):
        return "mtplt({})".format(self.line)

    def __str__(self):
        line=self.line.strip()
        line=line.strip("% ")
        if len(line)==0:
            return ""
        if self.type == matlab_to_python_line_transmogrification.HEADER:    
            return "\n\n# {}\n\n".format(line)
        if self.type == matlab_to_python_line_transmogrification.COMMENT:          
            badwords=["Key MATLAB commands", "www.mathworks.com"]
            for badword in badwords:
                if badword in line:
                    return ""
            if line.startswith("* "):
                line="\n"+line
            prefix=""
            postfix=""
            return "{}{}{}".format(prefix,line,postfix)
        if self.type == matlab_to_python_line_transmogrification.CODE:
            line=self.line
            # print("C. "+line)
            return "{}".format(line)
        return ""

def convert_m_script_to_ipython_notebook(m_file):
    nb = nbf.new_notebook()
    
    try:
        example, section = os.path.splitext(os.path.basename(m_file))[0].split("_")
    except:
        print(os.path.basename(m_file))
        print(os.path.splitext(os.path.basename(m_file))[0])
        print(os.path.splitext(os.path.basename(m_file))[0].split("_"))
        raise
    title = "{} - {}".format(re.sub(r"(\w)([A-Z])", r"\1 \2", example),
                             re.sub(r"(\w)([A-Z])", r"\1 \2", section))
    slug = "{}-{}".format(re.sub(r"(\w)([A-Z])", r"\1_\2", example.lower()),
                             re.sub(r"(\w)([A-Z])", r"\1_\2", section.lower()))
    with open (m_file, "r") as fid:
        lines=fid.readlines()
    
    last_mtplt=None
    for line in lines:
        mtplt = matlab_to_python_line_transmogrification(line)
        if last_mtplt is None:
            running_block=""           
            notebook_cells=list()
        else:
            if mtplt.cell_type != last_mtplt.cell_type:
                if last_mtplt.cell_type == 3:
                    cell_tmp = nbf.new_markdown_cell(running_block.strip())
                if last_mtplt.cell_type == 2:
                    cell_tmp = nbf.new_code_cell(running_block.strip())
                notebook_cells.append(cell_tmp)
                running_block=str_tmp=str(mtplt)
                last_mtplt = mtplt
                continue
        str_tmp=str(mtplt)
        if len(str_tmp) == 0:
            continue
        if running_block.endswith("\n"):
            running_block=(running_block+str_tmp)
        else:
            running_block=(running_block+" "+str_tmp)
        last_mtplt = mtplt
        

        
    ## Start Cells
    nb["cells"].append(nbf.new_markdown_cell(r"""<div id="toc"></div>"""))
        
    ## Body cells
    for notebook_cell in notebook_cells:
        cleaned_cell=cell_cleaner_upperer(notebook_cell)
        nb["cells"].append(cleaned_cell)
        
    ## End Cells
    nb["cells"].append(nbf.new_code_cell(r"""%%javascript
$.getScript('https://kmahelona.github.io/ipython_notebook_goodies/ipython_notebook_toc.js')"""))

    nikola_metadata=dict()
    nikola_metadatas=dict()
    nikola_metadata["author"]=""
    nikola_metadata["category"]=example
    nikola_metadata["date"]=""
    nikola_metadata["description"]=""
    nikola_metadata["link"]=""
    nikola_metadata["slug"]=slug
    nikola_metadata["tags"]=section
    nikola_metadata["title"]=title
    nikola_metadata["type"]=""
    nikola_metadatas["nikola"]=nikola_metadata
    nb["metadata"]=nikola_metadatas
    return nb